In [59]:
import copy
import random
import time
from collections import Counter

import numpy as np
import pandas as pd

from graphviz import Digraph
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

import warnings
warnings.filterwarnings('ignore')

In [60]:
# https://www.kaggle.com/datasets/sujithmandala/easiest-diabetes-classification-dataset
df_class = pd.read_csv('../cl_dataset.csv')
df_class

,Age,Gender,BMI,Blood Pressure,FBS,HbA1c,Family History of Diabetes,Smoking,Diet,Exercise,Diagnosis
0,45,Male,25,Normal,100,5.7,No,No,Healthy,Regular,No
1,55,Female,30,High,120,6.4,Yes,Yes,Poor,No,Yes
2,65,Male,35,High,140,7.1,Yes,Yes,Poor,No,Yes
3,75,Female,40,High,160,7.8,Yes,Yes,Poor,No,Yes
4,40,Male,20,Normal,80,5.0,No,No,Healthy,Regular,No
...,...,...,...,...,...,...,...,...,...,...,...
123,17,Female,15,Normal,100,5.7,No,Yes,Poor,No,Yes
124,22,Male,19,Normal,120,6.4,No,Yes,Poor,No,Yes
125,27,Female,24,High,140,7.1,No,Yes,Poor,No,Yes
126,32,Male,29,High,160,7.8,No,Yes,Poor,No,Yes


In [61]:
# https://www.kaggle.com/datasets/mirichoi0218/insurance
df_reg = pd.read_csv('../reg_dataset.csv')
df_reg

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [62]:
def introduce_missing_values(df, missing_percent=0.02, exclude_column='column'):
    """
    Вводит пропущенные значения в DataFrame.

    Параметры:
    df (pd.DataFrame): Исходный DataFrame.
    missing_percent (float): Процент значений, которые будут заменены на NaN.
    exclude_column (str): Название столбца, который будет исключен из процесса ввода пропущенных значений.

    Возвращает:
    pd.DataFrame: DataFrame с введенными пропущенными значениями.
    """

    df = df.copy()

    n_rows, n_cols = df.shape

    total_values = n_rows * n_cols

    n_missing = max(1, int(total_values * missing_percent))

    exclude_index = df.columns.get_loc(exclude_column)

    for _ in range(n_missing):
        i = random.randint(0, n_rows - 1)
        j = random.randint(0, n_cols - 1)

        while j == exclude_index:
            j = random.randint(0, n_cols - 1)

        df.iat[i, j] = np.nan

    return df

In [63]:
df_class_missing = introduce_missing_values(df_class, missing_percent=0.08, exclude_column='Diagnosis')
df_reg_missing = introduce_missing_values(df_reg, missing_percent=0.08, exclude_column='charges')

In [64]:
def minmax_scaler(df, cols_to_scale):
    """
    Масштабирует указанные столбцы DataFrame с использованием Min-Max нормализации.

    Параметры:
    df (pd.DataFrame): Исходный DataFrame.
    cols_to_scale (list): Список столбцов, которые нужно масштабировать.

    Возвращает:
    pd.DataFrame: DataFrame с масштабированными столбцами.
    """

    df = df.copy()

    for col in cols_to_scale:
        col_min = df[col].min()
        col_max = df[col].max()

        if col_max - col_min != 0:
            df[col] = (df[col] - col_min) / (col_max - col_min)
        else:
            df[col] = 0.5

    return df

In [65]:
df_class_clean_norm = minmax_scaler(
    df_class_missing, 
    cols_to_scale=["Age", "BMI", "FBS", "HbA1c"]
)

In [66]:
df_reg_clean_norm = minmax_scaler(
    df_reg_missing, 
    cols_to_scale=["age", "bmi", "children", "charges"]
)

In [67]:
def entropy(labels):
    """
    Вычисляет энтропию для заданного набора меток.

    Параметры:
    labels: Набор меток.

    Возвращает:
    float: Значение энтропии.
    """
    total = len(labels)
    counts = Counter(labels)
    ent = 0.0
    # вычисляем энтропию для каждой метки
    for _, cnt in counts.items():
        p = cnt / total  # вероятность метки
        # учитываем только положительные вероятности, чтобы избежать логарифма от нуля
        ent -= p * np.log2(p) if p > 0 else 0
    return ent

In [68]:
def donskoy_criterion(labels):
    """
    Вычисляет критерий Донского для заданного набора меток.

    Параметры:
    labels (list or array-like): Набор меток.

    Возвращает:
    float: Значение критерия Донского.
    """
    total = len(labels)
    counts = Counter(labels)
    res = 1.0
    for _, cnt in counts.items():
        p = cnt / total  # вероятность метки
        res -= p**2  # уменьшаем результат на квадрат вероятности
    return res

In [69]:
def mse_criterion(y_values):
    possible_values = np.unique(y_values)
    mse_list = [np.mean((y_values - y)**2) for y in possible_values]
    return min(mse_list)

In [70]:
def information_gain(current_labels, split_labels_list, criterion_function):
    parent_criterion = criterion_function(current_labels)
    total = len(current_labels)
    weighted_sum = 0.0
    for child_labels in split_labels_list:
        child_len = len(child_labels)
        weighted_sum += (child_len / total) * criterion_function(child_labels)
    return parent_criterion - weighted_sum

In [71]:
class DecisionNode:
    def __init__(self, 
                feature_name=None,
                threshold=None,
                left=None,
                right=None,
                leaf_value=None):
        self.feature_name = feature_name
        self.threshold = threshold
        self.left = left
        self.right = right
        self.leaf_value = leaf_value
        
    def is_leaf(self):
        return self.leaf_value is not None

In [72]:
def build_tree_id3(X, y, features, criterion_function, depth=0, max_depth=None, is_regression=False):
    """
    Строит дерево решений с использованием алгоритма ID3.

    Параметры:
    X (pd.DataFrame): Матрица признаков.
    y (pd.Series): Вектор меток.
    features (list): Список признаков для рассмотрения.
    criterion_function (callable): Функция критерия для вычисления информационного прироста.
    depth (int): Текущая глубина дерева.
    max_depth (int): Максимальная глубина дерева.
    is_regression (bool): Флаг, указывающий, является ли задача регрессией.

    Возвращает:
    DecisionNode: Корневой узел дерева решений.
    """

    # если все метки одинаковы, возвращаем листовой узел с этой меткой
    if len(set(y)) == 1:
        return DecisionNode(leaf_value=y.iloc[0])

    # если признаки закончились или достигнута максимальная глубина, возвращаем листовой узел
    if len(features) == 0 or (max_depth is not None and depth >= max_depth):
        if is_regression:
            return DecisionNode(leaf_value=np.mean(y))
        else:
            return DecisionNode(leaf_value=Counter(y).most_common(1)[0][0])

    best_gain = -1
    best_feature = None
    best_threshold = None
    best_splits = None

    # проходим по каждому признаку
    for feat in features:
        unique_values = X[feat].dropna().unique()  # исключаем NaN из уникальных значений

        # если признак числовой
        if X[feat].dtype in [float, int]:
            if len(unique_values) == 0:
                continue  # все значения пропущены
            threshold_candidates = [np.mean(unique_values)]
        else:
            threshold_candidates = list(unique_values) + [np.nan]

        # проходим по каждому кандидату на порог
        for t in threshold_candidates:
            if X[feat].dtype in [float, int]:
                left_mask = X[feat] <= t
                right_mask = X[feat] > t
                left_mask = left_mask | X[feat].isna()
            else:
                left_mask = X[feat] == t
                right_mask = X[feat] != t
                left_mask = left_mask | X[feat].isna()

            left_labels = y[left_mask]
            right_labels = y[right_mask]

            if len(left_labels) == 0 or len(right_labels) == 0:
                continue  # некорректный сплит

            # вычисляем информационный прирост для текущего разбиения
            current_gain = information_gain(y, [left_labels, right_labels], criterion_function)
            if current_gain > best_gain:
                best_gain = current_gain
                best_feature = feat
                best_threshold = t
                best_splits = (left_mask, right_mask)

    # если лучший прирост меньше порога, возвращаем листовой узел
    if best_gain < 1e-6:
        if is_regression:
            return DecisionNode(leaf_value=np.mean(y))
        else:
            return DecisionNode(leaf_value=Counter(y).most_common(1)[0][0])

    left_mask, right_mask = best_splits
    left_node = build_tree_id3(X[left_mask], y[left_mask], features, criterion_function,
                            depth+1, max_depth, is_regression)
    right_node = build_tree_id3(X[right_mask], y[right_mask], features, criterion_function,
                            depth+1, max_depth, is_regression)

    # возвращаем узел решения с лучшим признаком и порогом
    return DecisionNode(feature_name=best_feature,
                        threshold=best_threshold,
                        left=left_node,
                        right=right_node)

In [73]:
def predict_one_id3(x, node):
    """
    Предсказывает метку для одного примера с использованием дерева решений ID3.

    Параметры:
    x (pd.Series): Пример для предсказания.
    node (DecisionNode): Текущий узел дерева решений.

    Возвращает:
    any: Предсказанная метка.
    """

    # если текущий узел является листом, возвращаем его значение
    if node.is_leaf():
        return node.leaf_value

    feature = node.feature_name
    threshold = node.threshold
    value = x[feature]

    # если значение признака пропущено, идем в левое поддерево
    if pd.isna(value):
        return predict_one_id3(x, node.left)

    # если порог является числом
    if isinstance(threshold, float) or isinstance(threshold, int):
        if value <= threshold:
            return predict_one_id3(x, node.left)
        else:
            return predict_one_id3(x, node.right)
    else:
        # если порог является категориальным значением
        if value == threshold:
            return predict_one_id3(x, node.left)
        else:
            return predict_one_id3(x, node.right)

In [74]:
def predict_id3(X, tree_root):
    """
    Предсказывает метки для всех примеров в матрице признаков с использованием дерева решений ID3.

    Параметры:
    X (pd.DataFrame): Матрица признаков.
    tree_root (DecisionNode): Корневой узел дерева решений.

    Возвращает:
    np.array: Массив предсказанных меток.
    """

    preds = []
    
    # проходим по каждому примеру в матрице признаков
    for i in range(len(X)):
        x = X.iloc[i]  # извлекаем текущий пример
        preds.append(predict_one_id3(x, tree_root))  # предсказываем метку для текущего примера

    # возвращаем массив предсказанных меток
    return np.array(preds)

In [75]:
df_class_final = df_class_clean_norm.copy()
df_class_final["Diagnosis"] = df_class_final["Diagnosis"].apply(lambda x: 1 if x == "Yes" else 0)

In [76]:
def encode_categorical_cols(df):
    for col in df.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    return df

In [77]:
df_class_final = encode_categorical_cols(df_class_final)

In [78]:
X_class = df_class_final.drop("Diagnosis", axis=1)
y_class = df_class_final["Diagnosis"]

In [79]:
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_class, y_class, test_size=0.3, random_state=42, stratify=y_class
)

In [80]:
start_time = time.time()
tree_entropy = build_tree_id3(X_train_c, y_train_c, features=X_train_c.columns,
                            criterion_function=entropy, max_depth=None, is_regression=False)
time_entropy = time.time() - start_time

In [81]:
start_time = time.time()
tree_donskoy = build_tree_id3(X_train_c, y_train_c, features=X_train_c.columns,
                            criterion_function=donskoy_criterion, max_depth=None, is_regression=False)
time_donskoy = time.time() - start_time

In [82]:
y_pred_entropy = predict_id3(X_test_c, tree_entropy)
y_pred_donskoy = predict_id3(X_test_c, tree_donskoy)

In [83]:
def classification_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    return acc, prec, rec, f1

In [84]:
acc_e, prec_e, rec_e, f1_e = classification_metrics(y_test_c, y_pred_entropy)
print(f"Accuracy:  {acc_e:.3f}")
print(f"Precision: {prec_e:.3f}")
print(f"Recall:    {rec_e:.3f}")
print(f"F1-score:  {f1_e:.3f}")
print(f"Time:      {time_entropy:.6f} sec\n")

acc_d, prec_d, rec_d, f1_d = classification_metrics(y_test_c, y_pred_donskoy)
print(f"Accuracy:  {acc_d:.3f}")
print(f"Precision: {prec_d:.3f}")
print(f"Recall:    {rec_d:.3f}")
print(f"F1-score:  {f1_d:.3f}")
print(f"Time:      {time_donskoy:.6f} sec\n")

Accuracy:  0.821
Precision: 0.750
Recall:    0.333
F1-score:  0.462
Time:      0.111018 sec

Accuracy:  0.821
Precision: 0.750
Recall:    0.333
F1-score:  0.462
Time:      0.087565 sec



In [85]:
clf_sklearn = DecisionTreeClassifier(criterion="entropy", random_state=42, max_depth=None)  
start_time = time.time()
clf_sklearn.fit(X_train_c, y_train_c)
time_sklearn = time.time() - start_time

In [86]:
y_pred_sklearn = clf_sklearn.predict(X_test_c)
acc_s, prec_s, rec_s, f1_s = classification_metrics(y_test_c, y_pred_sklearn)

In [87]:
print(f"Accuracy:  {acc_s:.3f}")
print(f"Precision: {prec_s:.3f}")
print(f"Recall:    {rec_s:.3f}")
print(f"F1-score:  {f1_s:.3f}")
print(f"Time:      {time_sklearn:.6f} sec\n")

Accuracy:  0.949
Precision: 1.000
Recall:    0.778
F1-score:  0.875
Time:      0.002330 sec



In [88]:
df_reg_final = df_reg_clean_norm.copy()

In [89]:
df_reg_final = encode_categorical_cols(df_reg_final)

In [90]:
X_reg = df_reg_final.drop("charges", axis=1)
y_reg = df_reg_final["charges"]

In [91]:
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)

In [92]:
start_time = time.time()
tree_reg = build_tree_id3(X_train_r, y_train_r, features=X_train_r.columns,
                        criterion_function=mse_criterion, max_depth=None, is_regression=True)
time_reg_id3 = time.time() - start_time

In [93]:
y_pred_reg_id3 = predict_id3(X_test_r, tree_reg)

In [94]:
mse_id3 = mean_squared_error(y_test_r, y_pred_reg_id3)
r2_id3 = r2_score(y_test_r, y_pred_reg_id3)

In [95]:
print(f"MSE:  {mse_id3:.6f}")
print(f"R^2:  {r2_id3:.6f}")
print(f"Time: {time_reg_id3:.6f} sec\n")

MSE:  0.021217
R^2:  0.432076
Time: 11.640545 sec



In [96]:
reg_sklearn = DecisionTreeRegressor(criterion="squared_error", max_depth=None, random_state=42)
start_time = time.time()
reg_sklearn.fit(X_train_r, y_train_r)
time_sklearn_reg = time.time() - start_time

In [97]:
y_pred_reg_sklearn = reg_sklearn.predict(X_test_r)
mse_sklearn = mean_squared_error(y_test_r, y_pred_reg_sklearn)
r2_sklearn = r2_score(y_test_r, y_pred_reg_sklearn)

In [98]:
print(f"MSE:  {mse_sklearn:.6f}")
print(f"R^2:  {r2_sklearn:.6f}")
print(f"Time: {time_sklearn_reg:.6f} sec\n")

MSE:  0.013279
R^2:  0.644536
Time: 0.002199 sec



In [99]:
def prune_tree(node, X_val, y_val, is_regression=False):
    """
    Обрезка дерева решений с использованием валидационного набора данных.

    Параметры:
    node (DecisionNode): Текущий узел дерева решений.
    X_val (pd.DataFrame): Валидационная матрица признаков.
    y_val (pd.Series): Валидационный вектор меток.
    is_regression (bool): Флаг, указывающий, является ли задача регрессией. По умолчанию False.

    Возвращает:
    DecisionNode: Обрезанный узел дерева решений.
    """

    if node.is_leaf():
        return node

    # если валидационный набор пуст, возвращаем текущий узел
    if len(X_val) == 0 or len(y_val) == 0:
        return node

    # если левый дочерний узел существует, то рекурсивно обрезаем его
    if node.left is not None:
        feat = node.feature_name
        threshold = node.threshold
        if pd.isna(threshold):
            mask_left = X_val[feat].isna()
        elif isinstance(threshold, (float, int)):
            mask_left = (X_val[feat] <= threshold) | X_val[feat].isna()
        else:
            mask_left = (X_val[feat] == threshold) | X_val[feat].isna()
        X_val_left = X_val[mask_left]
        y_val_left = y_val[mask_left]
        node.left = prune_tree(node.left, X_val_left, y_val_left, is_regression)

    # если правый дочерний узел существует, то рекурсивно обрезаем его
    if node.right is not None:
        feat = node.feature_name
        threshold = node.threshold
        if pd.isna(threshold):
            mask_right = ~X_val[feat].isna()
        elif isinstance(threshold, (float, int)):
            mask_right = X_val[feat] > threshold
        else:
            mask_right = X_val[feat] != threshold
        X_val_right = X_val[mask_right]
        y_val_right = y_val[mask_right]
        node.right = prune_tree(node.right, X_val_right, y_val_right, is_regression)

    # предсказываем метки до обрезки
    preds_before = predict_id3(X_val, node)
    if is_regression:
        error_before = mean_squared_error(y_val, preds_before)
    else:
        acc_before = accuracy_score(y_val, preds_before)
        error_before = 1 - acc_before

    # сохраняем текущие дочерние узлы и признак
    left_backup = node.left
    right_backup = node.right
    feature_backup = node.feature_name
    threshold_backup = node.threshold

    # определяем значение листа
    if is_regression:
        leaf_value = np.mean(y_val) if len(y_val) > 0 else 0
    else:
        leaf_value = Counter(y_val).most_common(1)[0][0] if len(y_val) > 0 else 0

    # превращаем текущий узел в лист
    node.left = None
    node.right = None
    node.feature_name = None
    node.threshold = None
    node.leaf_value = leaf_value

    # предсказываем метки после обрезки
    preds_after = predict_id3(X_val, node)
    if is_regression:
        error_after = mean_squared_error(y_val, preds_after)
    else:
        acc_after = accuracy_score(y_val, preds_after)
        error_after = 1 - acc_after

    # если ошибка увеличилась, то восстанавливаем исходные дочерние узлы и признак
    if error_after > error_before:
        node.left = left_backup
        node.right = right_backup
        node.feature_name = feature_backup
        node.threshold = threshold_backup
        node.leaf_value = None

    return node

In [100]:
def print_classification_metrics(name, tree, X_test, y_test):
    print(f"\n{name})")

    y_pred = predict_id3(X_test, tree)
    acc, prec, rec, f1 = classification_metrics(y_test, y_pred)

    pruned_tree = prune_tree(copy.deepcopy(tree), X_test, y_test, is_regression=False)

    y_pred_pruned = predict_id3(X_test, pruned_tree)
    acc_pruned, prec_pruned, rec_pruned, f1_pruned = classification_metrics(y_test, y_pred_pruned)

    metrics_data = {
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score'],
        'Before Pruning': [acc, prec, rec, f1],
        'After Pruning': [acc_pruned, prec_pruned, rec_pruned, f1_pruned]
    }

    metrics_df = pd.DataFrame(metrics_data)
    metrics_df.set_index('Metric', inplace=True)

    print(metrics_df)

    return pruned_tree

In [101]:
tree_entropy_pruned = print_classification_metrics("Classification Tree, entropy", tree_entropy, X_test_c, y_test_c)
tree_donskoy_pruned = print_classification_metrics("Classification Tree, Donskoy", tree_donskoy, X_test_c, y_test_c)


Classification Tree, entropy)
           Before Pruning  After Pruning
Metric                                  
Accuracy         0.820513       0.974359
Precision        0.750000       1.000000
Recall           0.333333       0.888889
F1-score         0.461538       0.941176

Classification Tree, Donskoy)
           Before Pruning  After Pruning
Metric                                  
Accuracy         0.820513       0.974359
Precision        0.750000       1.000000
Recall           0.333333       0.888889
F1-score         0.461538       0.941176


In [102]:
tree_reg_pruned = prune_tree(copy.deepcopy(tree_reg), X_test_r, y_test_r, is_regression=True)
y_pred_reg_pruned = predict_id3(X_test_r, tree_reg_pruned)
mse_id3_pruned = mean_squared_error(y_test_r, y_pred_reg_pruned)
r2_id3_pruned = r2_score(y_test_r, y_pred_reg_pruned)

print(f"MSE before pruning: {mse_id3:.6f}, after: {mse_id3_pruned:.6f}")
print(f"R^2 before pruning: {r2_id3:.6f}, after: {r2_id3_pruned:.6f}")

MSE before pruning: 0.021217, after: 0.008563
R^2 before pruning: 0.432076, after: 0.770781


In [103]:
def visualize_tree(tree, name="tree"):
    """
    Визуализирует дерево решений с использованием Graphviz.

    Параметры:
    tree (DecisionNode): Корневой узел дерева решений.
    name (str): Имя графа. По умолчанию "tree".

    Возвращает:
    Digraph: Объект графа Graphviz.
    """

    dot = Digraph(name=name, format="png")
    node_counter = [0]

    def add_node(node, parent=None, edge_label=""):
        """
        Рекурсивно добавляет узлы и ребра в граф.

        Параметры:
        node (DecisionNode): Текущий узел дерева решений.
        parent (str): Идентификатор родительского узла. По умолчанию None.
        edge_label (str): Метка ребра. По умолчанию пустая строка.
        """

        # если текущий узел является листом, то добавляем его в граф
        if node.is_leaf():
            node_id = str(node_counter[0])
            dot.node(node_id, label=f"Leaf\nValue: {node.leaf_value}", shape="box", style="filled", color="lightgrey")
            if parent is not None:
                dot.edge(str(parent), node_id, label=edge_label)
            node_counter[0] += 1
            return

        # добавляем текущий узел в граф
        node_id = str(node_counter[0])
        dot.node(node_id, label=f"{node.feature_name} <= {node.threshold:.2f}")
        if parent is not None:
            dot.edge(str(parent), node_id, label=edge_label)
        node_counter[0] += 1

        # рекурсивно добавляем дочерние узлы
        add_node(node.left, parent=node_id, edge_label="Yes")
        add_node(node.right, parent=node_id, edge_label="No")

    # начинаем добавление узлов с корневого узла
    add_node(tree)
    return dot

In [104]:
dot_entropy = visualize_tree(tree_entropy, name="Entropy_Before_Pruning")
dot_entropy.render("Entropy_Before_Pruning", "../assets", cleanup=True)

'../assets/Entropy_Before_Pruning.png'

In [105]:
dot_entropy_pruned = visualize_tree(tree_entropy_pruned, name="Entropy_After_Pruning")
dot_entropy_pruned.render("Entropy_After_Pruning", "../assets", cleanup=True)

'../assets/Entropy_After_Pruning.png'

In [106]:
dot_donskoy = visualize_tree(tree_donskoy, name="Donskoy_Before_Pruning")
dot_donskoy.render("Donskoy_Before_Pruning", "../assets", cleanup=True)

'../assets/Donskoy_Before_Pruning.png'

In [107]:
dot_donskoy_pruned = visualize_tree(tree_donskoy_pruned, name="Donskoy_After_Pruning")
dot_donskoy_pruned.render("Donskoy_After_Pruning", "../assets", cleanup=True)

'../assets/Donskoy_After_Pruning.png'

In [108]:
dot_regression = visualize_tree(tree_reg, name="Regression_Before_Pruning")
dot_regression.render("Regression_Before_Pruning", "../assets", cleanup=True)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.471129 to fit


'../assets/Regression_Before_Pruning.png'

In [109]:
dot_regression_pruned = visualize_tree(tree_reg_pruned, name="Regression_After_Pruning")
dot_regression_pruned.render("Regression_After_Pruning", "../assets", cleanup=True)

'../assets/Regression_After_Pruning.png'